## Soft-Module performance check

In [1]:
import sys
sys.path.append("..")
import torchrl.policies as policies
import torchrl.networks as networks
from torchrl.utils import get_params
from torchrl.algo import MTSAC
import gym
from metaworld_utils.meta_env import get_meta_env, generate_single_mt_env
import numpy as np
import torch
import pprint

%load_ext autoreload
%autoreload 2

In [3]:
SEED = 11
params = get_params("../meta_config/mt10/modular_4_4_2_128_reweight_rand.json")

# create policy
device = torch.device("cuda:0")
#snapshot_path = "../log/starter/mt10/11/model/model_pf_best.pth"
#params['net']['base_type']=networks.MLPBase
#pf = policies.ModularGuassianGatedCascadeCondContPolicy(
#        input_shape=env.observation_space.shape[0],
#        em_input_shape=(np.prod(example_embedding.shape)),
#        output_shape=2 * env.action_space.shape[0],
#        **params['net'])
#pf.load_state_dict(torch.load(snapshot_path, map_location='cpu'))
#pf.to(device)
#pf.eval()
#env.eval()
#
#print("Module:", pf.named_modules)

In [4]:
# make mtenv
meta_env, cls_dicts, cls_args = get_meta_env( params['env_name'], params['env'], params['meta_env'])
tasks = list(cls_dicts.keys())
example_embedding = meta_env.active_task_one_hot
print("Obs_space:" , meta_env.observation_space)
print("Act_space:", meta_env.action_space)


Obs_space: Box(9,)
Act_space: Box(4,)


In [56]:
from sklearn.cluster import KMeans
import numpy as np

X = np.array([[1, 2, 2], [1, 4, 2], [1, 0, 3],
              [10, 2, 6], [10, 4, 5], [10, 0, 11]])
kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(X)
print(kmeans.labels_)
print(kmeans.cluster_centers_)
print(kmeans.inertia_/X.shape[0])

[1 1 1 0 0 2]
[[10.          3.          5.5       ]
 [ 1.          2.          2.33333333]
 [10.          0.         11.        ]]
1.8611111111111114


In [4]:
#choose a single task to evaluate on.
pprint.pprint(tasks)
env_id = tasks[8]
env_args = {
            "task_cls": cls_dicts[env_id],
            "task_args": cls_args[env_id],
            "env_rank": 8,
            "num_tasks": meta_env.num_tasks,
            "max_obs_dim": np.prod(meta_env.observation_space.shape),
            "env_params": params["env"],
            "meta_env_params": params["meta_env"]
        }


['reach-v1',
 'push-v1',
 'pick-place-v1',
 'door-v1',
 'drawer-open-v1',
 'drawer-close-v1',
 'button-press-topdown-v1',
 'ped-insert-side-v1',
 'window-open-v1',
 'window-close-v1']


In [21]:
#eval for timesteps
qf = networks.FlattenModularGatedCascadeCondNet(
        input_shape=meta_env.observation_space.shape[0] + meta_env.action_space.shape[0],
        em_input_shape=np.prod(example_embedding.shape),
        output_shape=1,
        **params['net'])




In [14]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np

p = np.array([0.5, 0.5, 0.0, 0.0])
q = np.array([0.25, 0.5, 0.25, 0.0])

eps = 1.0e-9

print(np.log2(q+eps))
def KLDiv(p, q):
    return np.sum(p * (np.log2(q+eps) - np.log2(p + eps)))

KLDiv(p, q)

[ -1.99999999  -1.          -1.99999999 -29.89735285]


-0.49999999855730487

In [119]:
# AFS
xs = torch.Tensor([[1, 2, 0.2, 0.1],
                   [-2, 1, 0.2, 0.1],
                   [1, 2, 0.2, 0.1],
                   [-2, 1, 0.2, 0.1]
                   ])
ws = torch.Tensor([0.4, 0.05, 0.6, 0.05])
ws = torch.softmax(ws, -1).unsqueeze(-1)
print(ws)
xs.requires_grad = True; ws.requires_grad=True
from torch.distributions import Normal
##dis = Normal(mu, sigma)
##action, z = dis.sample()
##log_prob = dis.log_prob(
##    action
##)
num_samples = 10


mu, sigma = torch.split(torch.sum(ws * xs, dim=0), 2)
print(mu, sigma)
dist = Normal(mu, sigma)
samples = dist.rsample((num_samples, ))

log_probs = dist.log_prob(samples)
print(log_probs)
dist.scale.retain_grad()
dist.loc.retain_grad()
loss = log_probs.sum()
loss.backward()
g = ws.grad

print(dist.loc.grad)
print(dist.scale.grad)
H = g@g.transpose(0, 1)/num_samples
print(xs.grad)
del ws, xs

tensor([[0.2754],
        [0.1941],
        [0.3364],
        [0.1941]])
tensor([-0.1645,  1.6118], grad_fn=<SplitBackward0>) tensor([0.2000, 0.1000], grad_fn=<SplitBackward0>)
tensor([[ 0.4839,  1.0281],
        [ 0.0355,  1.2168],
        [ 0.6884,  1.3834],
        [ 0.6309,  0.5829],
        [ 0.6901,  1.1042],
        [ 0.1278,  1.3686],
        [ 0.6883, -3.1957],
        [ 0.5765, -0.3208],
        [ 0.6172,  1.2779],
        [ 0.4589,  0.6360]], grad_fn=<SubBackward0>)
tensor([0., 0.])
tensor([ -50.0000, -100.0000])
tensor([[  0.0000,   0.0000, -13.7711, -27.5423],
        [  0.0000,   0.0000,  -9.7044, -19.4087],
        [  0.0000,   0.0000, -16.8201, -33.6402],
        [  0.0000,   0.0000,  -9.7044, -19.4087]])


In [5]:
import torchrl.policies as policies
import torchrl.networks as networks
SEED = 11
params['net']['base_type']=networks.MLPBase
pf = policies.ModularGuassianSelectCascadeContPolicy(
        input_shape=9,
        em_input_shape=(np.prod(example_embedding.shape)),
        output_shape=2 * meta_env.action_space.shape[0],
        **params['net'])
pf.to(device)

qf = networks.FlattenModularSelectCascadeCondNet(
        input_shape=meta_env.observation_space.shape[0] + meta_env.action_space.shape[0],
        em_input_shape=np.prod(example_embedding.shape),
        output_shape=1,
        **params['net'])
qf.to(device)
pf.train()
qf.train()
rews = []
rew = 0

env = generate_single_mt_env(**env_args)
env.seed(SEED)

# test env
eval_ob = env.reset()
try:
    for i in range(10):
        embedding_input = torch.zeros(env.num_tasks)
        embedding_input[env_args["env_rank"]] = 1
        embedding_input = embedding_input.unsqueeze(0).to(device)
        eval_ob = torch.Tensor( eval_ob ).to(device).unsqueeze(0)

        # policy
        out = pf.explore( eval_ob, embedding_input, return_weights = True)
        act = out["action"]

        # q
        q_pred = qf([eval_ob, act.unsqueeze(0)], embedding_input)
        act = act.detach().cpu().numpy()

        logits = out["general_weights"]
        eval_ob, r, done, info = env.step( act )
        rew += r
        rews.append(rew)
        done = info["success"]
finally:
    env.close()

print(pf)

ModularGuassianSelectCascadeContPolicy(
  (base): MLPBase(
    (fc0): Linear(in_features=9, out_features=400, bias=True)
    (fc1): Linear(in_features=400, out_features=400, bias=True)
  )
  (em_base): MLPBase(
    (fc0): Linear(in_features=10, out_features=400, bias=True)
  )
  (module_0_0): Linear(in_features=400, out_features=128, bias=True)
  (module_0_1): Linear(in_features=400, out_features=128, bias=True)
  (module_0_2): Linear(in_features=400, out_features=128, bias=True)
  (module_0_3): Linear(in_features=400, out_features=128, bias=True)
  (module_1_0): Linear(in_features=128, out_features=128, bias=True)
  (module_1_1): Linear(in_features=128, out_features=128, bias=True)
  (module_1_2): Linear(in_features=128, out_features=128, bias=True)
  (module_1_3): Linear(in_features=128, out_features=128, bias=True)
  (module_2_0): Linear(in_features=128, out_features=128, bias=True)
  (module_2_1): Linear(in_features=128, out_features=128, bias=True)
  (module_2_2): Linear(in_featur